# StaTexNet - I honestly don't know whether this does correct things

## Dependencies & Hyperparameters

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data
from torch.utils.data import DataLoader
import torch.nn as nn
import sys
import matplotlib.pyplot as plt
import numpy as np
sys.path.append('/home/gridsan/ckoevesdi/.local/lib/python3.9/site-packages/')
from pytorch_metric_learning import losses

sys.path.append('/home/gridsan/ckoevesdi/PooledStatisticsMetamers/poolstatmetamer/')
import utils.statnetencoder as sne
import importlib
import imp
import torch.nn.functional as F

#!wget https://raw.githubusercontent.com/pytorch/pytorch/master/torch/utils/collect_env.py

#sys.path.append(r'C:\Users\chris\Documents\MIT\Statistics_analysis_code\PyTorchSteerablePyramid')
sys.path.append('/home/gridsan/ckoevesdi/PyTorchSteerablePyramid/')
import steerable
import steerable.utils as utils
from steerable.SCFpyr_PyTorch import SCFpyr_PyTorch

torch.manual_seed(16)

#use GPU 2
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [2]:
#hyperparams
num_epochs = 50
batch_size = 20
crop_size = 128 
num_stats = 50 
optimizer_type='adam'
#optimizer_type='sgd'
learning_rate = 0.001
num_crops = 5 #changed this to four

multistat_penalty = 0
sparsity_penalty = 0.01
entropic_penalty = 0

#dataset location
#dtd_folder = '/gridsan/ckoevesdi/data/dtd_torch/dt/'
dtd_folder = 'home/gridsan/ckoevesdi/data/OT/dtd_torch/dtd/'

## Define DataLoader

In [3]:
loading_transforms = torchvision.transforms.Compose([#transforms.CenterCrop(size=300),
                                                    #transforms.RandomRotation(degrees=180),
                                                    transforms.Grayscale(), 
                                                    #transforms.TenCrop(size=crop_size),
                                                    #transforms.RandomRotation(degrees=[0,90,180,270]),
                                                    transforms.RandomVerticalFlip(p=0.5), #Commented out
                                                    transforms.RandomHorizontalFlip(p=0.5), #Commented out
                                                    transforms.FiveCrop(size=crop_size), #changed back to fivecrop
                                                    #transforms.functional.vflip(),
                                                    #transforms.functional.hflip(),
                                                    transforms.Lambda(lambda crops: torch.stack([transforms.PILToTensor()(crop) for crop in crops])),
                                                    transforms.ConvertImageDtype(torch.float32)])
                                                    #transforms.PILToTensor()])

#use training set for now
dtd_dataset = torchvision.datasets.DTD(root='/home/gridsan/ckoevesdi/data/OT/dtd_torch/', split='train', partition=1, 
                                       transform=loading_transforms, target_transform=None,
                                       download=False) #ah das datenset muss so aussehen wie es auf der website auch ist, deswegen kann man auch download false machen

sampler = data.RandomSampler(dtd_dataset)

dtd_dataloader = DataLoader(dtd_dataset, 
                            sampler=sampler,
                            batch_size=batch_size, 
                            shuffle=False)

tensor2pil_transform = transforms.ToPILImage()
print(len(dtd_dataset))

1879


In [4]:
import utils.brucenet as bn
brucy = bn.BruceNet(pooling_region_size=1e20, pyramid_params=False, dummy_img = torch.zeros(20,
                                     1,
                                     128,
                                     128)).to(device)


Creating solver: pyramid=UBBBBL_6 pooling=WholeImagePooling()


/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


## Checking which statistics really are the same

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Read the image

sys.path.append('/home/gridsan/ckoevesdi/video_pyramids/Imagestotest/')

#### IMAGE INPUT
image = cv2.imread('/home/gridsan/ckoevesdi/video_pyramids/Imagestotest/cracked_0004.jpg')

# Convert the image to grayscale
x = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
x = x/255.0
x_expanded = np.expand_dims(x, axis=0)
y = np.stack([x_expanded,x_expanded])
y_t = torch.from_numpy(y).to(dtype=torch.float32, device=device)
print(y_t.shape)

#### IMAGE OUTPUT
image2 = cv2.imread('/home/gridsan/ckoevesdi/video_pyramids/Imagestotest/cracked_0004_output.jpg')

# Convert the image to grayscale
x2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
x2 = x2/255.0
x2_expanded = np.expand_dims(x2, axis=0)
y2 = np.stack([x2_expanded,x2_expanded])
y2_t = torch.from_numpy(y2).to(dtype=torch.float32, device=device)

print(y2_t.shape)

z = brucy(y_t)
print(y_t)

In [ ]:
z1 = brucy(y_t)
z2 = brucy(y2_t)
print(z2)

#ordered_stats_labels_pretty = [' '.join([''.join(str(item)) for item in row if item not in ['',None]]) for row in stat_labels]
#print(ordered_stats_labels_pretty)

## Checking the distance between crops from same and from different image

In [8]:
if(True):
    for j, (texture_batch, labels) in enumerate(dtd_dataloader):
        print(texture_batch.shape)
        output = texture_batch[0].to(device)
        x = brucy(output)
        print(x.shape)
        #print(output.shape)
        output = torch.flatten(output, start_dim=0, end_dim=1)
        #print(output[j].squeeze().shape)
        #print(output.shape)
        expanded_labels = torch.repeat_interleave(labels, repeats=5, dim=0).to(device)
        #print(expanded_labels)
        print(x.shape)
        #fig, axes = plt.subplots(4, 5, figsize=(12, 10))
        #for j, ax in enumerate(axes.flat):
        #    ax.imshow(output[j].squeeze().cpu(), cmap='gray')  
        #    ax.set_title(f"Label: {expanded_labels[j]}")
        #    ax.axis('off')
        #plt.show()

        if j >= 2: 
            break


torch.Size([20, 5, 1, 128, 128])
torch.Size([5, 150])
torch.Size([5, 150])
torch.Size([20, 5, 1, 128, 128])
torch.Size([5, 150])
torch.Size([5, 150])
torch.Size([20, 5, 1, 128, 128])
torch.Size([5, 150])
torch.Size([5, 150])


In [9]:
m = x[1,:]
print(m)
m2 = x[2,:]
print(m)

tensor([ 3.1188e+00,  5.8007e-01,  4.0919e-02,  8.0725e-01,  4.6354e-01,
         2.7517e-01,  1.1201e+00,  6.3135e-01,  3.6666e-01,  1.5494e+00,
         8.6487e-01,  4.9778e-01,  2.1088e+00,  1.1658e+00,  6.6601e-01,
         2.7702e+00,  1.5074e+00,  8.5088e-01,  8.9115e-01,  1.7766e-01,
         9.4249e-02,  2.2064e-02,  1.1215e+00,  2.5165e-01,  1.1359e-01,
         2.4344e-02,  1.0981e+00,  2.8786e-01,  1.5740e-01,  3.1814e-02,
         7.8608e-01,  1.2580e-01,  5.4116e-02,  1.6262e-02,  8.2671e-02,
         7.1143e-02,  5.8561e-02,  1.0913e-01,  6.6748e-02,  7.8489e-02,
         9.8389e-01,  2.2002e-01,  1.7930e-01,  1.7653e-02,  1.2308e+00,
         2.2618e-01,  1.4270e-01,  4.1203e-02,  1.3359e+00,  3.4356e-01,
         2.3872e-01,  4.1744e-02,  7.7688e-01,  1.0423e-01,  7.0237e-02,
         2.0502e-02,  7.7644e-02,  6.6072e-02,  5.6152e-02,  1.1097e-01,
         5.7315e-02,  7.8774e-02,  1.8669e+00,  5.3379e-01,  4.6156e-01,
         3.9432e-02,  2.0226e+00,  4.0017e-01,  2.9

In [7]:
l = m
l2 = m2
print(l)

tensor([ 3.0137e+00,  5.0777e-01,  3.4296e-03,  7.1583e-01,  3.6377e-01,
         1.8610e-01,  1.0034e+00,  5.0855e-01,  2.5958e-01,  1.3926e+00,
         7.0297e-01,  3.5766e-01,  1.8966e+00,  9.5220e-01,  4.8329e-01,
         2.4796e+00,  1.2364e+00,  6.2878e-01,  3.3167e-01,  6.7510e-02,
         5.7115e-02,  1.4529e-03,  3.5113e-01,  3.1489e-02,  2.6969e-02,
         2.2738e-03,  3.9437e-01,  7.7429e-02,  6.5110e-02,  2.1895e-03,
         2.9339e-01,  2.4840e-02,  2.1070e-02,  2.0169e-03,  1.6121e-02,
         9.0356e-03,  1.4963e-02,  1.8486e-02,  1.2351e-02,  1.6867e-02,
         7.5064e-01,  1.6671e-01,  1.4823e-01,  7.4956e-03,  7.5115e-01,
         8.4568e-02,  6.5529e-02,  1.6544e-02,  8.6061e-01,  1.9042e-01,
         1.6191e-01,  1.1981e-02,  6.3577e-01,  6.6853e-02,  5.5573e-02,
         1.3987e-02,  4.3055e-02,  2.9400e-02,  3.9496e-02,  4.8488e-02,
         3.2325e-02,  4.4052e-02,  1.6505e+00,  4.5024e-01,  3.9269e-01,
         1.1711e-02,  1.4021e+00,  1.9038e-01,  1.4

In [ ]:
cos = torch.nn.CosineSimilarity(dim=0)
similarity12 = cos(m, m2).item()
similarity13 = cos(m, l).item()
similarity23 = cos(m2, l).item()
print(similarity12)
print(similarity13)
print(similarity23)

In [10]:
distance12 = torch.norm(m-m2).item()
distance13 = torch.norm(m-l).item()
distance23 = torch.norm(m2-l).item()
print(distance12)
print(distance13)
print(distance23)

2.3814940452575684
2.582104444503784
3.4768214225769043


In [ ]:
array1 = m
array2 = m2
array3 = l

# Combine tensors into a single tensor
combined = torch.stack([array1, array2, array3])
print(combined)

# Normalize
mean = torch.mean(combined, dim=0)
std = torch.std(combined, dim=0)
std[std == 0] = 1  # prevent division by zero
scaled_arrays = (combined - mean) / std
#print(scaled_arrays)
m = scaled_arrays[0,:]
m2= scaled_arrays[1,:]
l = scaled_arrays[2,:]

distance12 = torch.norm(m-m2).item()
distance13 = torch.norm(m-l).item()
distance23 = torch.norm(m2-l).item()
print(distance12)
print(distance13)
print(distance23)

In [ ]:
print(m2)

## Test Dataloader

## Define Model & Optimizer

In [ ]:
class TextureClassifier(nn.Module):
    def __init__(self, num_classes):
        super(TextureClassifier, self).__init__()
        
        self.fc1 = nn.Linear(150, 128)
        self.bn1 = nn.BatchNorm1d(128)
        
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.dropout(x, 0.5)
        
        x = F.relu(self.bn2(self.fc2(x)))
        x = F.dropout(x, 0.5)
        
        x = self.fc3(x)
        return x


In [ ]:
model = TextureClassifier(47)  # replace `number_of_classes` with your actual number
model = model.to(device)
statnet_model = sne.StatNetEncoder(img_size=(crop_size,crop_size),
                                   batch_size=batch_size,
                                   num_stats=num_stats,
                                   device=device)
statnet_model.to(device)
stat_labels = statnet_model.getsstatlabels(device)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# Training loop
num_epochs = 20
best_val_loss = float('inf')

In [ ]:
#Second try
from torch.utils.data import SubsetRandomSampler

validation_split = 0.2
shuffle_dataset = True
random_seed = 42

# Create data indices for training and validation splits
dataset_size = len(dtd_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

train_indices, val_indices = indices[split:], indices[:split]

# Creating data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(dtd_dataset, batch_size=batch_size, sampler=train_sampler, shuffle = False)
validation_loader = DataLoader(dtd_dataset, batch_size=batch_size, sampler=valid_sampler, shuffle = False)


In [ ]:
for x, _ in train_loader:
    print(x.shape)
    break



In [ ]:
print('Starting Training:')
for i, epoch in enumerate(range(num_epochs)):
    running_loss = 0.0
    correct = 0
    total = 0
    for j, (texture_batch, labels) in enumerate(train_loader):
        # Move data to device and flatten it
        output = texture_batch.to(device)
        output = torch.flatten(output, start_dim=0, end_dim=1)
        
        # Expand labels to account for crops
        expanded_labels = torch.repeat_interleave(labels, repeats=5, dim=0).to(device)
        
        # Skip if batch is smaller than usual
        current_batch_size = output.shape[0] // num_crops
        if current_batch_size < batch_size:
            continue

        # Forward pass
        stats = statnet_model.encoder(output)
        print(stats[1,:])
        optimizer.zero_grad()
        outputs = model(stats)
        
        # Compute loss and backpropagate
        loss = criterion(outputs, expanded_labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        # Compute accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += expanded_labels.size(0)
        correct += (predicted == expanded_labels).sum().item()

    # Print statistics for this epoch
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}, Accuracy: {100 * correct / total}")


    # Initialize variables to keep track of correct predictions and total predictions
    total_correct = 0
    total_predictions = 0

    # Disable gradient computation to speed up validation
    with torch.no_grad():
        for j, (texture_batch, labels) in enumerate(validation_loader):
            # Prepare data
            output = texture_batch.to(device)
            output = torch.flatten(output, start_dim=0, end_dim=1)
            expanded_labels = torch.repeat_interleave(labels, repeats=5, dim=0).to(device)

            # Skip if the batch is not full-sized
            current_batch_size = output.shape[0] // num_crops
            if current_batch_size < batch_size:
                continue  # Skip this batch if it's smaller than the usual batch size

            # Forward pass through the models
            stats = statnet_model.encoder(output)
            outputs = model(stats)

            # Calculate predictions
            _, predicted = torch.max(outputs, 1)

            # Update counters
            total_correct += (predicted == expanded_labels).sum().item()
            total_predictions += expanded_labels.size(0)

    # Calculate final accuracy
    accuracy = 100 * total_correct / total_predictions
    print(f'Validation Accuracy: {accuracy}%')

    # Switch model back to training mode
    model.train()
    statnet_model.train()


## Run Training

In [ ]:
for epoch in range(num_epochs):
    model.train()
    
    # Training loop
    for j, (texture_batch, labels) in enumerate(train_loader):
        output = texture_batch[0].to(device)
        output = torch.flatten(output, start_dim=0, end_dim=1)
        texture_labels = torch.repeat_interleave(torch.arange(batch_size),num_crops)
        #apply random permutation
        perm = torch.randperm(batch_size * num_crops)
        output = output[perm]
        texture_labels = texture_labels[perm]
        current_batch_size = output.shape[0] // num_crops
        #if current_batch_size < batch_size:
            #continue  # Skip this batch if it's smaller than the usual batch size


        # Assuming stats is a PyTorch tensor of shape (batch_size, 150)
        #print(output.shape)
        stats = statnet_model.encoder(output)
        
        optimizer.zero_grad()
        #print(stats.shape)
        outputs = model(stats)
        #print(outputs.shape)
        #print(labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 1
    with torch.no_grad():
           for texture_batch, labels in validation_loader:
            output = texture_batch.to(device)
            labels = torch.tensor([label for label in labels for _ in range(num_crops)]).to(device)
            current_batch_size = output.shape[0] // num_crops

            #if current_batch_size < batch_size:
                #continue  # Skip this batch if it's smaller than the usual batch size
            
            stats = statnet_model.encoder(output)
            outputs = model(stats)
            loss = criterion(outputs, texture_labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += texture_labels.size(0)
            print(total)
            correct += (predicted == texture_labels).sum().item()

    avg_val_loss = val_loss / len(validation_loader)
    val_accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

In [ ]:
print('Starting Training:')
for i, epoch in enumerate(range(num_epochs)):
    for j, texture_batch in enumerate(dtd_dataloader):
        #grab texture batch and generate matching labels
        output = texture_batch[0].to(device)
        output = torch.flatten(output, start_dim=0, end_dim=1)
        texture_labels = torch.repeat_interleave(torch.arange(batch_size),num_crops)
        perm = torch.randperm(batch_size * num_crops)

        current_batch_size = output.shape[0] // num_crops
        
        if current_batch_size < batch_size:
            continue  # Skip this batch if it's smaller than the usual batch size

        output = output[perm]
        texture_labels = texture_labels[perm]

        #calculate stats
        stats_vector = statnet_model.forward(output) #statsvector torch.Size([batchsize*crops, 50])
        #print(texture_labels.shape)
        #print(stats_vector.shape)
        
        #loss definitions
        representation_loss = loss_func_contrastive(stats_vector, texture_labels)
        sparsity_loss = statnet_model.sparse_loss()
        loss = (representation_loss + 
                (sparsity_penalty * sparsity_loss))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        #optimizer.step()
        #optimizer.zero_grad()
        #loss.backward()

        print('*',end='')
        #training_loss.append(loss.item())      
        if(j%30==0):
            print(loss.item())
        if(j==100):
            break;
        training_representation_loss.append(representation_loss.item())
        training_sparsity_loss.append(loss.item())    
        training_loss.append(loss.item())
    num_total_epochs = num_total_epochs + 1
    print(f'Finished Epoch {i}. Loss at {loss}.')
    #print('Initial Weights:', statnet_model._w)    
    if(i%10==0):
        compressor_mat = statnet_model.w.T.data.cpu().numpy()
        plt.imshow(compressor_mat)
        plt.colorbar() 
        plt.show()
    torch.save(
            {'state_dict': statnet_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'training_loss':training_loss,
            'last_output':output,
            'last_statvec':stats_vector},
            f'./model_checkpoint_epoch_{i}.pth')
print('All Done!')
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
plt.plot(training_loss)

In [ ]:
compressor_mat = statnet_model.w.T.data.cpu().numpy()
plt.figure(figsize=(10,5))
plt.imshow(compressor_mat)
plt.colorbar()
print(compressor_mat)

In [ ]:
stat_sums = np.sum(np.abs(compressor_mat),axis=0) #how weighted is each of the 150 stats?
stat_index_array = np.argsort(stat_sums)[::-1] #get their importance order

ordered_stat_sums = stat_sums[stat_index_array]
ordered_stat_labels = np.array(stat_labels,dtype=object)[stat_index_array]
ordered_stats_labels_pretty = [' '.join([''.join(str(item)) for item in row if item not in ['',None]]) for row in ordered_stat_labels]

n=25
print(f'Most Important {n} Stats:')
for i in range(n):
    print(ordered_stat_sums[i],ordered_stats_labels_pretty[i])

#plot the most important stats for contrastive learning
fig, ax = plt.subplots()
im = ax.imshow(compressor_mat[:,stat_index_array[:n]].T, aspect='auto')
fig.colorbar(im)
# Show all ticks and label them with the respective list entries
top_stats = ordered_stats_labels_pretty[:n]
ax.set_yticks(np.arange(len(top_stats)), labels=top_stats)
#Rotate the tick labels and set their alignment.
#plt.setp(ax.get_yticklabels(), rotation=45, ha="right",
#         rotation_mode="anchor")
plt.tight_layout()
plt.show()    
    
#plot most important stats for contrastive learning
#plt.imshow(compressor_mat[:,stat_index_array[:n]].T)

print(f'Least Important {n} Stats:')
for i in range(1,n+1):
    print(ordered_stat_sums[-i],ordered_stats_labels_pretty[-i])
    
    
#plot the most important stats for contrastive learning
fig, ax = plt.subplots()
im = ax.imshow(compressor_mat[:,stat_index_array[-n:]].T, aspect='auto')
fig.colorbar(im)
# Show all ticks and label them with the respective list entries
bot_stats = ordered_stats_labels_pretty[-n:][::-1]
ax.set_yticks(np.arange(len(bot_stats)), labels=bot_stats)
#Rotate the tick labels and set their alignment.
#plt.setp(ax.get_yticklabels(), rotation=45, ha="right",
#         rotation_mode="anchor")
plt.tight_layout()
plt.show()    

In [ ]:
?list.insert

In [ ]:
sys.path.append('/home/gridsan/ckoevesdi/.local/lib/python3.8/site-packages/')
from tsne_torch import TorchTSNE as TSNE

#texture_labels
#output
#calculate stats
stats_vector = statnet_model.forward(output)
print(stats_vector.shape)

X = stats_vector  # shape (n_samples, d)
X_emb = TSNE(n_components=2, perplexity=10, n_iter=10000, verbose=True).fit_transform(X)  # returns shape (n_samples, 2)



In [ ]:
from tsne_torch import TorchTSNE as TSNE

#texture_labels
#output
#calculate stats
stats_vector = statnet_model.forward(output)
print(stats_vector.shape)

X = stats_vector  # shape (n_samples, d)
X_emb = TSNE(n_components=2, perplexity=10, n_iter=10000, verbose=True).fit_transform(X)  # returns shape (n_samples, 2)



In [ ]:


#plt.legend(handles=scatter.legend_elements()[0], labels=texture_labels, title="Textures")
texture_label_list = texture_labels.numpy()

#fig, ax = plt.subplots()
#scatter = ax.scatter(X_emb[:,0],X_emb[:,1],c=texture_label_list,label=texture_label_list)
#plt.legend()#handles=scatter.legend_elements()[0], labels=texture_label_list)

#plt.legend()
#plt.legend(handles=scatter.legend_elements()[0],title="texture")

#handles = [plt.plot([],color=sc.get_cmap()(sc.norm(c)),ls="", marker="o")[0] for c,l in clset ]
#labels = [l for c,l in clset]
#ax.legend(texture_label_list)

unique_textures = set(texture_label_list)

#fix, ax = plt.subplots()
plt.figure(figsize=(6,6))
for i, texture in enumerate(unique_textures):
    texture_index = np.where(texture_label_list == texture)[0]
    texture_index = np.where(texture_label_list==i)[0]
    plt.scatter(X_emb[texture_index,0],X_emb[texture_index,1],label=i)
plt.legend()
plt.title(f'T-sne embeddings: {num_crops} crops, {batch_size} textures')
plt.tight_layout()

for i, label in enumerate(texture_label_list):
    plt.annotate(label, (X_emb[i,0],X_emb[i,1] + 0.2))

#for i, txt in enumerate(texture_label_list):
#    plt.annotate(texture_label_list, (X_emb[:,0],X_emb[:,1]))
plt.tight_layout()

